In [1]:
# import statements
import numpy as np
import dicom
import cv2 
import matplotlib.pyplot as plt
from collections import defaultdict
import scipy.misc as misc
import sys
import os
sys.path.append("..")
from AlgoEngine.utils import getVolume, getContours, getImageBlock, convertROIToCTSpace, getMeanTargetDose
from General.testing_utils import getContourInputs
from AlgoEngine.similarity import getTDDistance

## Inputs to Function

We generate dose means for two patients- the query `UCLA_PR_5` and the historical patient `UCLA_PR_6`

In [2]:
base = '/home/radiation/RadiationTherapyDecisionSupport/data/'

In [3]:
StudyID = 'UCLA_PR_5'
structureset = dicom.read_file(base + StudyID + '/structureset.dcm')
dose_data = dicom.read_file(base + StudyID + "/dose.dcm")
dose_grid = np.array(dose_data.pixel_array)

x_spacing = np.array(dose_data.PixelSpacing[0]).astype(np.uint8)
y_spacing = np.array(dose_data.PixelSpacing[1]).astype(np.uint8)

In [4]:
dose_z = float(dose_data.ImagePositionPatient[2])
ctFilenames = [fl for fl in os.listdir(base + StudyID) if 'CT.' in fl]
ct_sample = dicom.read_file(base + StudyID + '/' + ctFilenames[0])
block_shape = (ct_sample.Rows, ct_sample.Columns, len(ctFilenames))

ct_coords = None

for i, fl in enumerate(ctFilenames):
    ct_struct = dicom.read_file(base + StudyID + '/' + fl)
    if(float(ct_struct.ImagePositionPatient[2]) == dose_z):
        ct_coords = ct_struct.ImagePositionPatient
        ct_spacing = ct_struct.PixelSpacing

In [5]:
x0 = float(dose_data.ImagePositionPatient[0])- float(ct_coords[0])
x0 = int(round(x0 / float(ct_spacing[0])))
y0 = float(dose_data.ImagePositionPatient[1])- float(ct_coords[1])
y0 = int(round(y0 / float(ct_spacing[1])))

DoseGridScaling = dose_data.DoseGridScaling
_, sopUID = getImageBlock(StudyID, base)

In [6]:
_, sop_ids = getImageBlock(StudyID, base)

ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(base, StudyID, ROI_NAME, excluding=[])
_, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)
query_dose_mean = getMeanTargetDose(ptv_roi_block, block_shape, dose_grid, DoseGridScaling, x0, y0, x_spacing, y_spacing, sop_ids)

In [7]:
StudyID = 'UCLA_PR_6'
structureset = dicom.read_file(base + StudyID + '/structureset.dcm')
dose_data = dicom.read_file(base + StudyID + "/dose.dcm")
dose_grid = np.array(dose_data.pixel_array)

x_spacing = np.array(dose_data.PixelSpacing[0]).astype(np.uint8)
y_spacing = np.array(dose_data.PixelSpacing[1]).astype(np.uint8)

In [8]:
dose_z = float(dose_data.ImagePositionPatient[2])
ctFilenames = [fl for fl in os.listdir(base + StudyID) if 'CT.' in fl]
ct_sample = dicom.read_file(base + StudyID + '/' + ctFilenames[0])
block_shape = (ct_sample.Rows, ct_sample.Columns, len(ctFilenames))

ct_coords = None

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(base + StudyID + '/' + fl).ImagePositionPatient[2]
    ct_struct = dicom.read_file(base + StudyID + '/' + fl)
    if(float(ct_struct.ImagePositionPatient[2]) == dose_z):
        ct_coords = ct_struct.ImagePositionPatient
        ct_spacing = ct_struct.PixelSpacing
                        

In [9]:
x0 = float(dose_data.ImagePositionPatient[0])- float(ct_coords[0])
x0 = int(round(x0 / float(ct_spacing[0])))
y0 = float(dose_data.ImagePositionPatient[1])- float(ct_coords[1])
y0 = int(round(y0 / float(ct_spacing[1])))

DoseGridScaling = dose_data.DoseGridScaling

In [17]:
_, sop_ids = getImageBlock(StudyID, base)

ROI_NAME = 'PTV'
block_shape, contour_data, image_orientation, image_position, pixel_spacing = getContourInputs(base, StudyID, ROI_NAME, excluding=[])
_, ptv_roi_block = getContours(block_shape, contour_data, image_orientation, image_position, pixel_spacing)
ptv_roi_block = convertROIToCTSpace(ptv_roi_block, image_position, sop_ids)
historical_dose_mean = getMeanTargetDose(ptv_roi_block, block_shape, dose_grid, DoseGridScaling, x0, y0, x_spacing, y_spacing, sop_ids)

## Function

In [11]:
dissimilarity = getTDDistance(query_dose_mean, historical_dose_mean)

## Testing Functions

We print the dissimilarity, and confirm that none of the PTV dose means are 0.

In [12]:
dissimilarity

0.12232474

In [13]:
query_dose_mean

0.12232474

In [18]:
historical_dose_mean

0.0